<a href="https://colab.research.google.com/github/Zurcaid/MachineLearning/blob/main/PatricLacouth/redes_convolucionais_com_tensorflow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Desafio

Vamos tentar criar em tempo real, um sistema que identifica objetos em tempo real.

**Etapas**

0. **FAÇA UMA CÓPIA DO NOTEBOOK PARA SUA CONTA**
1. Capturar imagens de pelo menos dois objetos utilizando a webcam.
2. Definir um modelo com camadas convolucionais usando TensorFlow
3. Treinar o modelo e verificar a acurácia.
4. Testar o modelo treinado.

# Execute a célula abaixo para definir a função que captura imagens

In [ ]:
from IPython.display import display, Javascript, clear_output, Image
from google.colab.output import eval_js
from base64 import b64decode
from pathlib import Path

def take_photo(filename='photo.jpg', quality=0.4, width=320, height=240):
  js = Javascript('''
    async function takePhoto(quality, width, height) {
      const div = document.createElement('div');
      const capture = document.createElement('button');
      capture.textContent = 'Capturar';
      div.appendChild(capture);

      const video = document.createElement('video');
      video.style.display = 'block';
      const stream = await navigator.mediaDevices.getUserMedia({video: true});

      document.body.appendChild(div);
      div.appendChild(video);
      video.srcObject = stream;
      await video.play();

      // Resize the output to fit the video element.
      google.colab.output.setIframeHeight(document.documentElement.scrollHeight, true);

      // Wait for Capture to be clicked.
      await new Promise((resolve) => capture.onclick = resolve);

      const canvas = document.createElement('canvas');
      canvas.width = width; // Set desired width
      canvas.height = height; // Set desired height
      const ctx = canvas.getContext('2d');
      ctx.drawImage(video, 0, 0, width, height); // Resize the video feed to canvas
      stream.getVideoTracks()[0].stop();
      div.remove();
      return canvas.toDataURL('image/jpeg', quality);
    }
  ''')
  display(js)
  data = eval_js(f'takePhoto({quality}, {width}, {height})')
  binary = b64decode(data.split(',')[1])
  with open(filename, 'wb') as f:
    f.write(binary)
  return filename


# O código abaixo permite coletar as imagens para o treinamento.
## Colete o máximo que puder para cada classe, pelo menos 5 imagens de cada

In [ ]:
label = input('Informe a classe da imagem: ')
label = label
Path('dados/'+label).mkdir(parents=True, exist_ok=True)
contador = 0
continuar = 's'
while continuar == 's':
  contador += 1
  filename = take_photo(filename = 'dados/'+label+'/'+label+'_'+str(contador)+'.jpg', quality=0.5, width=96,height=96)
  print('Imagem salva em {}'.format(filename))
  display(Image(filename))
  continuar = input('Deseja continuar (s/n): ').lower()
  clear_output()


In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import os
import PIL
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential

batch_size = 1
img_height = 96
img_width = 96



In [ ]:
#divisão do conjunto de treinamento
train_ds = tf.keras.utils.image_dataset_from_directory(
  'dados',
  validation_split=0.2,
  subset="training",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size)

In [ ]:
#divisão do conjunto de validação
val_ds = tf.keras.utils.image_dataset_from_directory(
  'dados',
  validation_split=0.2,
  subset="validation",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size)


In [ ]:
class_names = train_ds.class_names
print(class_names)

In [ ]:
#definição do modelo

num_classes = len(class_names)

model = Sequential([
  layers.Rescaling(1./255, input_shape=(img_height, img_width, 3)),
  layers.RandomFlip(), #data augmentation
  layers.RandomRotation(factor=0.5), #data augmentation
  layers.Conv2D(16, 3, padding='same', activation='relu'), #camanda conv1
  layers.MaxPooling2D(), #redução de dimensionalidade
  layers.Conv2D(32, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Conv2D(64, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Dropout(0.2),
  layers.Flatten(),
  layers.Dense(128, activation='relu'),
  layers.Dense(num_classes, activation='softmax')
])

model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [ ]:
#treinamento do modelo
epochs = 100
history = model.fit(
  train_ds,
  validation_data=val_ds,
  epochs=epochs
)

In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

loss = history.history['loss']
val_loss = history.history['val_loss']

epochs_range = range(epochs)

plt.figure(figsize=(8, 8))
plt.plot(epochs_range, acc, label='Acurácia treinamento')
plt.plot(epochs_range, val_acc, label='Acurácia validação')
plt.legend(loc='upper right')
plt.title('Acurácia')
plt.grid()
plt.show()

# Código para testar com uma nova imagem

In [ ]:
Path('teste').mkdir(parents=True, exist_ok=True)
filename = take_photo(filename='teste/teste.jpg', quality=0.4)
display(Image(filename))

img = tf.keras.utils.load_img(
    filename, target_size=(img_height, img_width)
)
img_array = tf.keras.utils.img_to_array(img)
img_array = tf.expand_dims(img_array, 0)

predictions = model.predict(img_array)
score = tf.nn.softmax(predictions[0])

print(
    "Essa imagem provavelmente é {} com {:.2f} porcento de confiança."
    .format(class_names[np.argmax(score)], 100 * np.max(score))
)

# **Baixar as imagens para criar um modelo do EDGE IMPULSE**

[Edge Impulse](https://edgeimpulse.com/)

In [ ]:
!zip -r /content/dados.zip /content/dados

In [ ]:
from google.colab import files
files.download("/content/dados.zip")